In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 

c:\Users\mandy\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#xdf = pd.read_csv('/content/drive/MyDrive/ML/Datasets/Chinese_segmentation/as_training.utf8', header=None, names=['raw_words'])
#xfor_vocab = open('/content/drive/MyDrive/ML/Datasets/Chinese_segmentation/as_training.utf8')

#testdf = pd.read_csv('/content/drive/MyDrive/ML/Datasets/Chinese_segmentation/as_test.utf8', header=None)

In [6]:
xdf = pd.read_csv(r'as_training.utf8', header=None, names=['raw_words'])

In [7]:
def spaceornot(line):
  splace = []
  listed = list(line)
  for i, char in enumerate(listed):
    if listed[i] == listed[-1]:
      splace.append(1)
    elif char == '\u3000':
      pass
    elif listed[i + 1] == '\u3000':
      splace.append(1)
    elif listed[i + 1] != '\u3000':
      splace.append(0)
  return splace

In [8]:
yraw = xdf.copy()

xtrain = pd.DataFrame(xdf['raw_words'].str.replace('\u3000',''))   #could ignore last punctuation

#tokening _input data
tokenizer = Tokenizer(char_level=True, oov_token='<OOV>')
        #dlist_sent = seriestolist(xtrain, 0)
tokenizer.fit_on_texts(xtrain['raw_words'])
word_index = tokenizer.word_index
xtrain['sequenced_sent'] = tokenizer.texts_to_sequences(xtrain['raw_words'])
X = pad_sequences(xtrain['sequenced_sent'], padding='post')

#y_actual
yraw['labels'] = yraw['raw_words'].map(spaceornot)
Y = pad_sequences(yraw['labels'], padding='post')


# alternatively, you can pass the padded arrays into the dataframe by converting it into a list using list()
"""
used data:
train_data: xseq
train_label: yactual
"""

'\nused data:\ntrain_data: xseq\ntrain_label: yactual\n'

In [9]:
val_split = 0.75
vsize = len(word_index)
sentlen = len(X[0,:])
em_dim = 256
epol = 1000
basize = 1024

#callbacks

In [10]:
#splitting dataset
spli = int(val_split * len(X))
trainX, testX = X[:spli], X[spli:]
trainY, testY = Y[:spli], Y[spli:]


#possible to split in ~model~.fit() function too

In [7]:
'''
Custom Loss
'''

class MaskedSequenceLoss(tf.keras.losses.Loss):
    def __init__(
        self,
        average_across_timesteps=False,
        average_across_batch=False,
        sum_over_timesteps=True,
        sum_over_batch=True,
        softmax_loss_function=None,
        name=None,
        reduction=None, # dummy arg so it can be used as custom object when loading saved model
    ):
        super().__init__()
        self.opts = {
            "average_across_timesteps": average_across_timesteps,
            "average_across_batch": average_across_batch,
            "sum_over_timesteps": sum_over_timesteps,
            "sum_over_batch": sum_over_batch,
            "softmax_loss_function": softmax_loss_function,
            "name": name,
        }
    
    def call(self, y_true, y_pred):
        return tfa.seq2seq.sequence_loss(y_pred, y_true,
                                         weights=tf.cast(y_pred._keras_mask, tf.float32) if hasattr(y_pred, "_keras_mask") else tf.ones(y_true.shape),
                                         **self.opts)

In [8]:
xin = Input((sentlen,))

x = Embedding(vsize, em_dim, input_length=sentlen, mask_zero = True)(xin) #ignore padded zeros -> mask_zero = True
x = Bidirectional(LSTM(256, return_sequences=True))(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)

x = Dense(128, activation='swish')(x)
x = Dropout(0.7)(x)
x = Dense(64, activation='swish')(x) 
x = Dropout(0.7)(x)
x = Dense(64, activation='swish')(x) 
x = Dropout(0.7)(x)

xout = Dense(2, activation='linear')(x)

segcl = Model(xin, xout)
segcl.compile(optimizer='adam', loss=MaskedSequenceLoss(), metrics=['accuracy'])
segcl.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 188)]             0         
                                                                 
 embedding (Embedding)       (None, 188, 256)          1557760   
                                                                 
 bidirectional (Bidirectiona  (None, 188, 512)         1050624   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 188, 256)         656384    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 188, 128)          32896     
                                                                 
 dropout (Dropout)           (None, 188, 128)          0     

In [9]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.1, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('./best_models', monitor='val_acc', save_best_only=True)
]

In [10]:
segcl.fit(
    trainX, 
    trainY,
    validation_data=(testX, testY),
    epochs = epol,
    batch_size = basize,
    callbacks = callbacks
)

#you can use validation_split = True to split the data into val and train, instead of manual splitting
#using tf.stack, you can pass the dataframe series into it directly
'''
segcl.fit(

)
'''


Epoch 1/1000


InternalError: Graph execution error:

Detected at node 'gradients/cond_grad/gradients/cond/CudnnRNNV3_grad/CudnnRNNBackpropV3' defined at (most recent call last):
    File "c:\Users\mandy\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\mandy\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\mandy\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\mandy\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\mandy\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\mandy\AppData\Local\Programs\Python\Python39\lib\site-packages\tornado\

In [2]:
segcl = tf.keras.models.load_model("8_saved_models/8_best_model_weights")
segcl.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, 188)]       0         
                                                                 
 dense_10 (Dense)            (None, None, 64)          12096     
                                                                 
 dense_11 (Dense)            (None, None, 64)          4160      
                                                                 
 dense_12 (Dense)            (None, None, 64)          4160      
                                                                 
 dense_13 (Dense)            (None, None, 64)          4160      
                                                                 
 dense_14 (Dense)            (None, None, 188)         12220     
                                                                 
Total params: 36,796
Trainable params: 36,796
Non-trainable

In [3]:
def splitter(sentences, text_split_indexes):
    dat = pd.DataFrame(sentences, columns=['unsplit'])
    splited = []
    for i, sentence in enumerate(sentences):
        split_sen = ''
        for n, sply in enumerate(list(text_split_indexes[i])[:len(sentence)]):
            split_sen += sentence[n]
            if sply == 1:
                split_sen += '\u3000'

        splited.append(split_sen)
    dat['splited'] = pd.DataFrame(splited)

    return dat

def display(dataframe):
    for i in range(len(dataframe.iloc[:,0])):
        print('original sentence:  ' + dataframe.iloc[i, 0])
        print('splitted sentence:  ' + dataframe.iloc[i, 1])
        print('\n')


def display_with_correct(dataframe, correct):
    for i in range(len(dataframe.iloc[:,0])):
        print('original sentence:  ' + dataframe.iloc[i, 0])
        print('splitted sentence:  ' + dataframe.iloc[i, 1])
        print('actually sentence:  ' + correct.iloc[i, 0])
        print('\n')

def entire(input):
    input_list = []
    input_list.append(input)
    token = tokenizer.texts_to_sequences(input_list)
    token = pad_sequences(token, padding='post', maxlen=sentlen)

    pred = segcl.predict(token)
    pred = tf.nn.softmax(pred)
    pred = np.argmax(pred, axis=-1)

    return display(splitter(input_list, pred))

In [4]:
input_sentences=[
    '生日快樂',
    '我的名字是',
    '今天的天氣很不可思議',
    '中文單詞',
    '雪花飄飄北風蕭蕭',
    'Google 的免費服務可即時在英語和 100 多種其他語言之間翻譯單詞、短語和網頁。'

]

In [11]:
input_sentences = list(xtrain.iloc[:10,0])

In [12]:
token_sentences = tokenizer.texts_to_sequences(input_sentences)
token_sentences = pad_sequences(token_sentences, padding='post', maxlen=sentlen)

text_predictions = segcl.predict(token_sentences)
text_predictions = tf.nn.softmax(text_predictions)
text_predictions = np.argmax(text_predictions, axis=-1)

org_data = splitter(input_sentences, text_predictions)

1/1 [==============================] - 11s 11s/step


TypeError: 'numpy.int64' object is not iterable

In [ ]:
display(org_data)

In [ ]:
entire('雪花飄飄北風蕭蕭')

In [ ]:
#done